In [4]:
import threading
import time
import os

def run_flask():
    os.system("python app.py")

# Run Flask in a separate thread
flask_thread = threading.Thread(target=run_flask, daemon=True)
flask_thread.start()

# The Flask app runs in the background, and you can continue using the notebook.


In [11]:
import numpy as np
import folium
import utm
import requests
import matplotlib.pyplot as plt
from scipy.interpolate import LinearNDInterpolator

def plot_easting_northing_matrix_in_google_map(eastings_northings_matrix, my_title="Location of Points on Google Maps", ref_lat_lon=None):
    """
    Converts Easting/Northing to Latitude/Longitude, calibrates the points, and plots them on an interactive Google Map.
    
    Parameters:
        eastings_northings_matrix (list of lists): Easting/Northing coordinates.
        my_title (str): Title for the plot.
        ref_lat_lon (tuple or None): Reference latitude/longitude (e.g., Meskel Square) to determine the UTM zone.
    """
    try:
        if ref_lat_lon and len(ref_lat_lon) == 2:
            example_easting, example_northing, zone, _ = utm.from_latlon(ref_lat_lon[0], ref_lat_lon[1])
        else:
            zone = 37  # Default UTM zone for Addis Ababa

        eastings = np.array(eastings_northings_matrix)[:, 0]
        northings = np.array(eastings_northings_matrix)[:, 1]

        # Calibration values for correcting positions
        given_easting = np.array([477504.6975, 482977.07875, 487741.8536])
        given_northing = np.array([980922.813, 992734.94275, 993586.1784])
        easting_calib_req = np.array([90.6484, 92.6484, 94.6484])
        northing_calib_req = np.array([204.2779, 210.2779, 208.2779])

        # Compute the mean values for interpolation
        this_mean_easting = np.mean(eastings)
        this_mean_northing = np.mean(northings)

        # Linear interpolation for calibration
        F_easting_calib_req = LinearNDInterpolator(list(zip(given_easting, given_northing)), easting_calib_req)
        F_northing_calib_req = LinearNDInterpolator(list(zip(given_easting, given_northing)), northing_calib_req)

        calib_value_eastings = F_easting_calib_req(this_mean_easting, this_mean_northing)
        calib_value_northings = F_northing_calib_req(this_mean_easting, this_mean_northing)

        precalib_eastings = eastings + calib_value_eastings
        precalib_northings = northings + calib_value_northings

        computed_lats, computed_lons = [], []
        for e, n in zip(precalib_eastings, precalib_northings):
            lat, lon = utm.to_latlon(e, n, zone, northern=True)
            computed_lats.append(lat)
            computed_lons.append(lon)

        # Close the plot shape
        computed_lats.append(computed_lats[0])
        computed_lons.append(computed_lons[0])

        # Plot on an interactive Google Map
        center_lat, center_lon = np.mean(computed_lats), np.mean(computed_lons)
        map_ = folium.Map(location=[center_lat, center_lon], zoom_start=17, tiles="OpenStreetMap")

        # Add markers for each point
        for lat, lon in zip(computed_lats[:-1], computed_lons[:-1]):
            folium.Marker([lat, lon], popup="Point").add_to(map_)

        # Draw polygon
        folium.PolyLine(list(zip(computed_lats, computed_lons)), color="blue", weight=2.5, opacity=1).add_to(map_)

        print("✅ Successfully plotted given coordinates on Google Map.")

        # Save as an HTML file and open in browser
        map_.save("map.html")
        import webbrowser
        webbrowser.open("map.html")

        # Open Google Maps with a route if a reference location is provided
        if ref_lat_lon:
            google_maps_url = f"https://www.google.com/maps/dir/{center_lat},{center_lon}/{ref_lat_lon[0]},{ref_lat_lon[1]}"
            webbrowser.open(google_maps_url)

    except Exception as e:
        print("❌ Failed to plot given coordinates on Google Map.")
        print(f"Error: {e}")



In [12]:
easting_northing_data = [
    [482000, 990500],
    [482500, 990800],
    [482200, 991000],
    [482000, 990500]  # Closing the polygon
]

ref_point = (9.0108, 38.7613)  # Example: Meskel Square
plot_easting_northing_matrix_in_google_map(easting_northing_data, "Land Plot", ref_point)


✅ Successfully plotted given coordinates on Google Map.


In [10]:
pip install utm

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
